<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C9-Fun_With_Exfiltration/Encrypting_And_Decrypting_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Fun with Exfiltration__*

Gaining access to a target network is only a part of the battle. To make use of your access, you want to be able to exfiltrate documents, spreadsheets, or other bits of data from the target system. Depending on the defense mechanisms in place, this last part of your attack can prove to be tricky.
There might be local or remote systems (or a combination of both) that work to validate processes that open remote connections as well as determine whether those processes should be able to send information or initiate connections outside of the internal network.

In this chapter, we'll create tools that enable you to exfiltrate encrypted data. First, we'll write a script to encrypt and decrypt files. We'll then use that script to encrypt information and transfer it from the system by using three methods: Email, file transfers, and posts to a web server. For each of these methods, we'll write both a platform-independent tool and a Windows-only tool.

For the Windows-only functions, we'll rely on the __PyWin32__ libraries we used in Chapter 8, especially the __win32com__ package. Windows __COM__ (Component Object Model) automation serves a number of practical uses, from interacting with network-based services to embedding a Microsoft Excel spreadsheet into your own application. All versions of Windows, beginning with XP, allow you to embed an Internet Explorer COM object into applications, and we'll take advantage of this ability in this chapter.

### *__Encrypting and Decrypting Files__*

We'll use the __pycryptodomex__ package for the encryption tasks. You can install it with this command:

```
$ pip install pycryptodomex
```

Now, open up __cryptor.py__ and let's import the libraries we'll need to get started:

In [ ]:
from Cryptodome.Cipher import AES, PKCS1_OAEP #[1]
from Cryptodome.PublicKey import RSA #[2]
from Cryptodome.Random import get_random_bytes
from io import BytesIO

import base64
import zlib

We'll create a hybrid encryption process, using symmetric and asymmetric encryption to get the best of both worlds. The __AES__ cipher is an example of symmetric encryption __[1]__: It's called __symmetric__ because it uses a single key for both encryption and decryption. It is very fast, and it can handle large amounts of text. That's the encryption method we will use to encrypt the information we want to exfiltrate.

We also import the __asymmetric__ __RSA__ cipher __[2]__, which uses a public key / private key technique. It relies on one key for the encryption (typically the public key) and the other for decryption (typically the private key). We will use this cipher to encrypt the single key used in the AES encryption. The asymmetric encryption is well suited to small bits of information, making it perfect for encrypting the AES key.

This method of using both types of encryption is called a __hybrid system__, and it's very common. For example, the __TLS__ communication between your browser and a web server involves a hybrid system.
Before we can begin encrypting or decrypting, we'll need to create public and private keys for the asymmetric RSA encryption. That is, we need to create an RSA key generation function. Let's start by adding a __generate__ function to __cryptor.py__:

In [ ]:
def generate():
    new_key = RSA.generate(2048)
    private_key = new_key.exportKey()
    public_key = new_key.publicKey().exportKey()

    with open('key.pri', 'wb') as f:
        f.write(private_key)

    with open('key.pub', 'wb') as f:
        f.write(public_key)

That's right, Python is so badass that we can do this in an handful of lines of code. This block of code outputs both a private and public key pair in the files named __key.pri__ and __key.pub__. Now let's create a small helper function so we can grab either the public or private key:

In [ ]:
def get_rsa_cipher(keytype):
    with open(f'key.{keytype}') as f:
        key = f.read()
    rsakey = RSA.importKey(key)
    return (PKCS1_OAEP.new(rsakey), rsakey.size_in_bytes())

We pass this function the key type (__pub__ or __pri__), read the corresponding file, and return the cipher object and the size of the RSA key in bytes.
Now that we've generated two keys and have a function to return an RSA cipher from the generated keys, let's go on with encrypting the data:

In [ ]:
def encrypt(plaintext):
    compressed_text = zlib.compress(plaintext) #[1]

    session_key = get_random_bytes(16) #[2]
    cipher_aes = AES.new(session_key, AES.MODE_EAX)
    ciphertext, tag = cipher_aes.encrypt_and_digest(compressed_text) #[3]

    cipher_rsa, _ = get_rsa_cipher('pub')
    encrypted_session_key = cipher_rsa.encrypt(session_key) #[4]

    msg_payload = encrypted_session_key + cipher_aes.nonce + tag + ciphertext #[5]
    encrupted = base64.encodebytes(msg_payload) #[6]
    return(encrypted)

We pass in the plaintext as bytes and compress it __[1]__. We then generate a random session key to be used in the AES cipher __[2]__ and encrypt the compressed plaintext using that cipher __[3]__. Now that the information is encrypted, we need to pass the session key as part of the returned payload, along with the ciphertext iself, so it can be decrypted on the other side. To add the session key, we encrypt it with the RSA key generated from the generated public key __[4]__. We put all the information we need to decrypt into one payload __[5]__, base64-encode it, and return the resulting encrypted string __[6]__.

Now let's fill out the __decrypt__ function:

In [ ]:
def decrypt(encrypted):
    encrypted_bytes = BytesIO(base64.decodebytes(encrypted)) #[1]
    cipher_rsa, keysize_in_bytes = get_rsa_cipher('pri')

    encrypted_session_key = encrypted_bytes.read(keysize_in_bytes) #[2]
    nonce = encrypted_bytes.read(16)
    tag = encrypted_bytes.read(16)
    ciphertext = encrypted_bytes.read()

    session_key = cipher_rsa.decrypt(encrypted_session_key) #[3]
    cipher_aes = AES.new(session_key, AES.MODE_EAX, nonce)
    decrypted = cipher_aes.decrypt_and_verify(ciphertext, tag) #[4]

    plaintext = zlib.decompress(decrypted) #[5]
    return plaintext

To decrypt, we reverse the steps from the __encrypt__ function. First, we base64-decode the string into bytes __[1]__. Then we read the encrypted session key, along with the other parameters we need to decrypt, from the encrypted byte string __[2]__. We decrypt the session key using the RSA private key __[3]__ and use that key to decrypt the message itself with the AES cipher __[4]__. Finally we decompress it into a plaintext byte string __[5]__ and return.

Next, this main block makes it easy to test the functions:

In [ ]:
if __name__ == '__main__':
    generate() #[1]

In one step, we generate the public and private keys __[1]__. We're simply calling the __generate__ function since we have to generate the keys before we can use them. Now we can edit the main block to use the keys:

In [ ]:
if __name__ == '__main__':
    plaintext = b'hey there you.'
    print(decrypt(encrypt(plaintext))) #[1]

After the keys are generated, we encrypt and then decrypt a small byte string and then print the result __[1]__.